In [1]:
import pandas as pd
import numpy as np

In [2]:
vcf = pd.read_pickle("../data/all_vcfs.pkl")
c = pd.read_csv("../data/clinical/clinical.csv").drop("Unnamed: 0", axis=1).rename(columns={"PTID":"Subject"})
img = pd.read_pickle('../data/MRI/mri_meta.pkl')[["img_array", "subject", "label"]]

In [4]:
# c = c.rename(columns = {"Group":"GROUP"})

In [4]:
img = img.rename(columns = {"subject":"Subject"})

In [5]:
c.columns, vcf.columns, img.columns

(Index(['Subject', 'RID', 'Phase', 'Group', 'PTGENDER_-4.0', 'PTGENDER_1.0',
        'PTGENDER_2.0', 'PTHOME_-4.0', 'PTHOME_1.0', 'PTHOME_2.0',
        ...
        'PHC_MEM_SE', 'PHC_VSP_SE', 'PHC_EXF', 'PHC_LAN', 'PTADDX', 'PHC_MEM',
        'PHC_LAN_SE', 'PTCOGBEG', 'PHC_VSP', 'PHC_Age_Cognition'],
       dtype='object', length=153),
 Index(['Subject',   'Group',     12116,     12117,     12118,     12119,
            12120,     12121,     12122,     12123,
        ...
           599531,    599532,    599533,    599534,    599535,    599536,
           599537,    599538,    599539,    599540],
       dtype='object', length=179668),
 Index(['img_array', 'Subject', 'label'], dtype='object'))

In [6]:
a = vcf.merge(c, on = ["Subject", "Group"]).merge(img, on = "Subject")

In [15]:
a["Subject"].value_counts()

006_S_0498    2
027_S_1045    1
007_S_1206    1
002_S_1155    1
036_S_1023    1
             ..
005_S_0546    1
014_S_0557    1
027_S_0644    1
016_S_0359    1
127_S_1419    1
Name: Subject, Length: 78, dtype: int64

In [13]:
a["Group"].value_counts()

0.0    46
1.0    33
Name: Group, dtype: int64

In [16]:
cols = list(set(a.columns) - set(["PTID", "label", "GROUP",
                                  "RID", "ID", "Group", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX", "Unnamed: 0"]))

In [18]:
X = a[cols]
y = a["Group"]

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [20]:
X_test[["Subject"]].to_csv("overlap_test_set.csv")

In [21]:
snp_cols = set(X_train.columns).intersection(set(vcf.columns))

In [23]:
X_train_snp = X_train[snp_cols]
X_test_snp = X_test[snp_cols]

/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/443268023.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_train_snp = X_train[snp_cols]
/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/443268023.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_test_snp = X_test[snp_cols]


In [24]:
img_cols = set(X_train.columns).intersection(set(img.columns))
print(len(img.columns))
print(len(img_cols))
X_train_img = X_train[img_cols]
X_test_img = X_test[img_cols]

3
2


/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/1154895174.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_train_img = X_train[img_cols]
/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/1154895174.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_test_img = X_test[img_cols]


In [25]:
clin_cols = set(X_train.columns).intersection(set(c.columns))
print(len(c.columns))
print(len(clin_cols))
X_train_clin = X_train[clin_cols]
X_test_clin = X_test[clin_cols]

153
150


/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/1500334310.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_train_clin = X_train[clin_cols]
/var/folders/fs/dzz16rpn4gb232kj3rp5q6sh0000gn/T/ipykernel_63398/1500334310.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_test_clin = X_test[clin_cols]


In [26]:
!pwd

/Users/flynnzhang/CMU/Fall24/ML_Healthcare/Project/MADDi-ML/preprocess_overlap


In [27]:
pd.DataFrame(X_train_snp).to_pickle("X_train_snp.pkl")
pd.DataFrame(X_test_snp).to_pickle("X_test_snp.pkl")
pd.DataFrame(y_train).to_pickle("y_train.pkl")
pd.DataFrame(y_test).to_pickle("y_test.pkl")

pd.DataFrame(X_train_clin).to_pickle("X_train_clinical.pkl")
pd.DataFrame(X_test_clin).to_pickle("X_test_clinical.pkl")

pd.DataFrame(X_train_img).to_pickle("X_train_img.pkl")
pd.DataFrame(X_test_img).to_pickle("X_test_img.pkl")

---------------------------------------------------------------------------------------------------------